In [5]:
import pandas as pd, numpy as np, json

In [6]:
eu_vt=json.loads(open('data/eu_allegiance.json','r').read())

In [ ]:
eu_allegiance=json.loads(open('data/eu_allegiance.json','r').read())

In [ ]:
records=json.loads(open('data/records.json','r').read())

In [ ]:
countries=sorted(eu_vt)

Allegiance

In [15]:
def get_allegiance_matrix(key,vt,allegiance):
    allegiance_matrix={}
    initvote={'Same':0,'Opposite':0,'Total':0}
    for j1 in vt:
        outcome=j1['outcome']
        name1=j1[key]
        if name1 not in allegiance_matrix:allegiance_matrix[name1]={}
        if outcome=='DA':
            for name2 in allegiance[j1['vote']]['DA']:
                if name2 not in allegiance_matrix[name1]:
                    allegiance_matrix[name1][name2]=dict(initvote)
                allegiance_matrix[name1][name2]['Total']+=1
                allegiance_matrix[name1][name2]['Same']+=1
            if 'NU' in allegiance[j1['vote']]:
                for name2 in allegiance[j1['vote']]['NU']:
                    if name2 not in allegiance_matrix[name1]:
                        allegiance_matrix[name1][name2]=dict(initvote)
                    allegiance_matrix[name1][name2]['Total']+=1
                    allegiance_matrix[name1][name2]['Opposite']+=1
        elif outcome=='NU':
            for name2 in allegiance[j1['vote']]['NU']:
                if name2 not in allegiance_matrix[name1]:
                    allegiance_matrix[name1][name2]=dict(initvote)
                allegiance_matrix[name1][name2]['Total']+=1
                allegiance_matrix[name1][name2]['Same']+=1
            if 'DA' in allegiance[j1['vote']]:
                for name2 in allegiance[j1['vote']]['DA']:
                    if name2 not in allegiance_matrix[name1]:
                        allegiance_matrix[name1][name2]=dict(initvote)
                    allegiance_matrix[name1][name2]['Total']+=1
                    allegiance_matrix[name1][name2]['Opposite']+=1
    for j in allegiance_matrix:
        for i in allegiance_matrix[j]:
            allegiance_matrix[j][i]['Same_perc']=np.round(allegiance_matrix[j][i]['Same']/allegiance_matrix[j][i]['Total'],3)
            allegiance_matrix[j][i]['Opposite_perc']=np.round(allegiance_matrix[j][i]['Opposite']/allegiance_matrix[j][i]['Total'],3)
    return allegiance_matrix

In [16]:
def clean_country(country):
    if country=='Atlantic Ocean area':return 'Atlantic Ocean Area'
    if country=='Baltic Sea area':return 'Baltic Sea Area'
    if country=='Former Yugoslav Republic of Macedonia':return 'Macedonia'
    if country=='Former Yugoslav Republic of Macedonia (FYROM)':return 'Macedonia'
    if country=='Kosovo under UNSCR 1244/1999':return 'Kosovo'
    if country=='Mediterranean Sea area':return 'Mediterranean Sea Area'
    if country=='Moldova, Republic':return 'Moldova'
    if country=='Montenegro, from 06/2006':return 'Montenegro'
    if country=='Serbia, from 06/2006':return 'Serbia'
    if country=='North Sea area':return 'North Sea Area'
    return country

In [20]:
def clean_topic(topic,k=3):
    return topic[:k]

In [21]:
def get_vote_topics(voteid):
    topics=set()
    for i in voteid_2_dossierid[voteid]:
        if 'subject' in i:
            for topic in i['subject']:
                topics.add(clean_topic(topic))
        elif 'sujet' in i:
            for topic in i['sujet']:
                topics.add(clean_topic(topic))
    return list(topics)

In [22]:
def get_vote_areas(voteid):
    on_countries=set()
    for i in voteid_2_dossierid[voteid]:
        if 'geographical_area' in i:
            for on_country in i['geographical_area']:
                on_countries.add(clean_country(on_country))
    return list(on_countries)

In [185]:
eu_allegiance_matrix={}
topic_types=['topic','area']
for country in countries:
    for year in sorted(eu_vt[country]):
        votes_by_topic={}
        votes_by_area={}
        for vote in eu_vt[country][year]:
            topics=get_vote_topics(vote['vote'])
            areas=get_vote_areas(vote['vote'])
            for topic in topics:
                if topic not in votes_by_topic:votes_by_topic[topic]=[]
                votes_by_topic[topic].append(vote)
            for area in areas:
                if area not in votes_by_area:votes_by_area[area]=[]
                votes_by_area[area].append(vote)
        votes_by_topic['All topics']=list(eu_vt[country][year])
        votes_by_area['All areas']=list(eu_vt[country][year])
        for topic_type in topic_types:
            if topic_type=='topic':
                sub_type=votes_by_topic
            else:
                sub_type=votes_by_area
            for topic in sub_type:
                for allegiance_type1 in ['name','group','party']:
                    for allegiance_type2 in ['name','group','party']:
                        dummy=get_allegiance_matrix(allegiance_type1,sub_type[topic],
                                eu_allegiance[country][year][allegiance_type2])
                        if dummy!={}:
                            if country not in eu_allegiance_matrix:
                                eu_allegiance_matrix[country]={}
                            if year not in eu_allegiance_matrix[country]:
                                eu_allegiance_matrix[country][year]={}
                            if topic_type not in eu_allegiance_matrix[country][year]:
                                eu_allegiance_matrix[country][year][topic_type]={}
                            if topic not in eu_allegiance_matrix[country][year][topic_type]:
                                eu_allegiance_matrix[country][year][topic_type][topic]={}
                            if allegiance_type1 not in eu_allegiance_matrix[country][year][topic_type][topic]:
                                eu_allegiance_matrix[country][year][topic_type][topic][allegiance_type1]={}
                            if allegiance_type2 not in eu_allegiance_matrix[country][year][topic_type][topic][allegiance_type1]:
                                eu_allegiance_matrix[country][year][topic_type][topic][allegiance_type1][allegiance_type2]={}
                            eu_allegiance_matrix[country][year][topic_type][topic][allegiance_type1][allegiance_type2]=dummy
            print(country,year,topic_type)

Hungary 2004 topic
Hungary 2004 area
Hungary 2005 topic
Hungary 2005 area
Hungary 2006 topic
Hungary 2006 area
Hungary 2007 topic
Hungary 2007 area
Hungary 2008 topic
Hungary 2008 area
Hungary 2009 topic
Hungary 2009 area
Hungary 2010 topic
Hungary 2010 area
Hungary 2011 topic
Hungary 2011 area
Hungary 2012 topic
Hungary 2012 area
Hungary 2013 topic
Hungary 2013 area
Hungary 2014 topic
Hungary 2014 area
Hungary 2015 topic
Hungary 2015 area
Hungary 2016 topic
Hungary 2016 area
Hungary 2017 topic
Hungary 2017 area
Hungary 2018 topic
Hungary 2018 area
Hungary 2019 topic
Hungary 2019 area
Joint 2004 topic
Joint 2004 area
Joint 2005 topic
Joint 2005 area
Joint 2006 topic
Joint 2006 area
Joint 2007 topic
Joint 2007 area
Joint 2008 topic
Joint 2008 area
Joint 2009 topic
Joint 2009 area
Joint 2010 topic
Joint 2010 area
Joint 2011 topic
Joint 2011 area
Joint 2012 topic
Joint 2012 area
Joint 2013 topic
Joint 2013 area
Joint 2014 topic
Joint 2014 area
Joint 2015 topic
Joint 2015 area
Joint 2016 t

Clusterings

In [186]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [187]:
def dict_2_matrix(matrix,key,party_labels=False):
    labels=sorted(matrix)
    slabels=[]
    for i in range(len(labels)):
        label=labels[i]
        if label in names:
            if party_labels:
                party=party_normalizer(names[label]['Constituencies'][0]['party'])
                group=party_normalizer(names[label]['Groups'][0]['groupid'])
                slabels.append(str(label)+u' | '+str(party)+' | '+str(group))
            else:
                slabels.append(label)
        else:
            slabels.append(label)

    #extend to square matrix
    inner_keys=matrix[sorted(matrix)[0]]
    inner_keys=sorted(inner_keys[sorted(inner_keys)[0]])
    for name1 in  labels:
        for name2 in labels:
            if name2 not in matrix[name1]:
                matrix[name1][name2]={i:0 for i in inner_keys}
    return np.array([[matrix[name1][name2][key] for name2 in sorted(matrix[name1])] for name1 in labels]),slabels

In [188]:
def hier_cluster(matrix,level,th=1,key='Same_perc',party_labels=False,method='single', metric='euclidean',criterion='distance'):
    X,labelList=dict_2_matrix(matrix[level][level],key,party_labels)
    if len(X)>1:
        try:
            linked = linkage(X, method=method,metric=metric)
        except:
            try:
                linked = linkage(X, method=method,metric='euclidean')
            except:
                linked = linkage(X, method='single',metric='euclidean')
        f=fcluster(linked, th, criterion)
        labelList=[labelList[i]+' | '+str(f[i]) for i in range(len(labelList))]
        return linked,labelList
    else:
        return [],[]

Exctract clusters

In [189]:
def get_unique_parent_node(nodes_children,node):
    if node in leafs:
        return node
    elif len(nodes_children[node])>1:
        return node
    else:
        return get_unique_parent_node(nodes_children,nodes_children[node][0])
def get_unique_parent(node,node_dict,unique_node_set,root):
    if node not in node_dict: 
        return root
    elif node_dict[node] in unique_node_set:
        return node_dict[node]
    else:
        return get_unique_parent(node_dict[node],node_dict,unique_node_set,root)

Save for VEGA

In [190]:
master_tree={}
nc_levels=10
key='Same_perc'
mpruned_nodes=[]
for country in countries:
    for year in eu_allegiance_matrix[country]:
        for topic_type in eu_allegiance_matrix[country][year]:
            print(country,year,topic_type)
            for topic in eu_allegiance_matrix[country][year][topic_type]:
                for allegiance in eu_allegiance_matrix[country][year][topic_type][topic]:
                    uid=country+year+allegiance+topic_type+topic
                    cluster_list=[]
                    clusterdummy={}
                    for nc in range(2,nc_levels):
                        hc,hlabels=hier_cluster(eu_allegiance_matrix[country][year][topic_type][topic],
                                                allegiance,nc,key,True,'complete','seuclidean','maxclust')
                        for i in hlabels:
                            hi=i.split('|')
                            name=hi[0].strip()
                            cluster_no=hi[-1].strip()
                            if name not in clusterdummy:
                                clusterdummy[name]={}
                                clusterdummy[name]['name']=name
                                clusterdummy[name]['cluster_level_'+str(nc_levels)]=name
                                clusterdummy[name]['country']=country
                                clusterdummy[name]['cluster_level_1']=country
                            clusterdummy[name]['cluster_level_'+str(nc)]='c'+str(nc)+str(cluster_no)
                    cluster_list=list(clusterdummy.values())
                    #construct tree
                    leafs=sorted(clusterdummy)
                    nodes=[{'name':country}]
                    nodes_done=set()
                    nodes_children={}
                    for i in cluster_list:
                        for cluster_level in range(2,nc_levels+1):
                            node=i['cluster_level_'+str(cluster_level)]
                            parent=i['cluster_level_'+str(cluster_level-1)]
                            if node not in nodes_done:
                                dummy={}
                                nodes_done.add(node)
                                dummy['name']=node
                                dummy['parent']=parent
                                if parent not in nodes_children:nodes_children[parent]=[]
                                nodes_children[parent].append(node)
                                nodes.append(dummy)
                    #get unique nodes
                    node_dict={i['name']:i['parent'] for i in nodes[1:]}
                    unique_nodes={}
                    for node in nodes_children:
                        unique_nodes[node]=get_unique_parent_node(nodes_children,node)
                    unique_node_set=set(unique_nodes.values()).union(set(leafs))
                    #prune
                    pruned_nodes=[]
                    for i in nodes:
                        dummy=i
                        name=i['name']
                        if 'parent' not in i:
                            pruned_nodes.append(i)
                        elif i['name'] in unique_node_set:
                            dummy['parent']=get_unique_parent(name,node_dict,unique_node_set,nodes[0]['name'])
                            if name in leafs:
                                if allegiance=='name':
                                    dummy['party']=party_normalizer(names[name]['Constituencies'][0]['party'])
                                    dummy['group']=party_normalizer(names[name]['Groups'][0]['groupid'])
                                    dummy['party2']=party_normalizer2(names[name]['Constituencies'][0]['party'])
                                    dummy['group2']=party_normalizer2(names[name]['Groups'][0]['groupid'])
                                else:
                                    dummy['party']=''
                                    dummy['group']=''
                                    dummy['party2']=''
                                    dummy['group2']=''
                                dummy['image']=get_photo(name,names,allegiance)     
                            pruned_nodes.append(dummy)
                    for i in pruned_nodes:
                        dummy=i
                        if 'party' in dummy:
                            dummy['partycolor']=get_link_color(dummy['party'])
                        if 'group' in dummy:
                            dummy['groupcolor']=get_link_color(dummy['group'])
                        dummy['country']=country
                        dummy['year']=year
                        dummy['topic_type']=topic_type
                        dummy['topic']=topic
                        dummy['allegiance']=allegiance
                        mpruned_nodes.append(dummy)

Hungary 2004 topic
Hungary 2004 area
Hungary 2005 topic
Hungary 2005 area
Hungary 2006 topic
Hungary 2006 area
Hungary 2007 topic
Hungary 2007 area
Hungary 2008 topic
Hungary 2008 area
Hungary 2009 topic
Hungary 2009 area
Hungary 2010 topic
Hungary 2010 area
Hungary 2011 topic
Hungary 2011 area
Hungary 2012 topic
Hungary 2012 area
Hungary 2013 topic
Hungary 2013 area
Hungary 2014 topic
Hungary 2014 area
Hungary 2015 topic
Hungary 2015 area
Hungary 2016 topic
Hungary 2016 area
Hungary 2017 topic
Hungary 2017 area
Hungary 2018 topic
Hungary 2018 area
Hungary 2019 topic
Hungary 2019 area
Joint 2004 topic
Joint 2004 area
Joint 2005 topic
Joint 2005 area
Joint 2006 topic
Joint 2006 area
Joint 2007 topic
Joint 2007 area
Joint 2008 topic
Joint 2008 area
Joint 2009 topic
Joint 2009 area
Joint 2010 topic
Joint 2010 area
Joint 2011 topic
Joint 2011 area
Joint 2012 topic
Joint 2012 area
Joint 2013 topic
Joint 2013 area
Joint 2014 topic
Joint 2014 area
Joint 2015 topic
Joint 2015 area
Joint 2016 t

In [191]:
open('ep/export/json/topics.json','w').write(json.dumps(mpruned_nodes))

50212122

In [193]:
save_local(mpruned_nodes,'topics.json')